In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
%run Features.ipynb

# Algoritmo

In [4]:
from sksurv.preprocessing import OneHotEncoder

features_v1 = OneHotEncoder().fit_transform(features_v1)
features_v2 = OneHotEncoder().fit_transform(features_v2)
features_v1.head()

,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,ref_hash,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_14,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23
941,4.0,1.0,0.0,73.0,8.0,2.0,0.0,0.0,17411113585415081,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0
401,254.0,33.0,11.0,0.0,42.0,108.0,104.0,39.0,18336104939494418,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
980,36.0,19.0,86.0,51.0,111.0,64.0,34.0,82.0,24003528264893641,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
913,318.0,0.0,0.0,2.0,81.0,177.0,61.0,37.0,28354923597842736,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0
505,32.0,2.0,0.0,0.0,29.0,4.0,1.0,1.0,31813139061954413,1.0,...,0.0,0.0,0.0,0.0,0.0,5.0,20.0,11.0,0.0,0.0


In [5]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis(alpha=0.0001)
estimator.fit(features_v1,label1_b)

CoxPHSurvivalAnalysis(alpha=0.0001, n_iter=100, tol=1e-09, verbose=0)

In [6]:
pd.Series(estimator.coef_, index=features_v1.columns)

cantidad_de_subastas_en_ventana         2.125217e-03
cantidad_subastas_Monday                1.660858e-04
cantidad_subastas_Tuesday               1.446177e-03
cantidad_subastas_Wednesday            -1.243722e-03
cantidad_subastas_Thursday              1.048771e-03
cantidad_subastas_Friday               -1.562976e-03
cantidad_subastas_Saturday             -2.260882e-03
cantidad_subastas_Sunday               -7.183369e-04
ref_hash                               -4.596436e-21
cantidad_de_clicks_en_ventana           7.359700e-02
cantidad_clicks_Monday                 -5.067374e-02
cantidad_clicks_Tuesday                 7.500156e-02
cantidad_clicks_Wednesday               1.683948e-01
cantidad_clicks_Thursday                2.120369e-01
cantidad_clicks_Friday                  4.166864e-02
cantidad_clicks_Saturday               -2.624522e-01
cantidad_clicks_Sunday                  1.056911e-02
SUP_D                                   6.017412e-01
SUP_C                                   1.2508

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [7]:
estimator.score(features_v2, label2_b)

0.6223929512494429

# KNN

In [580]:
df = label1.sort_values(ascending=True,by='device_id')
df.head()

,device_id,st,binary
32647,17411113585415081,68166.777507,True
8346,18336104939494418,5620.605960,True
34253,24003528264893641,72429.253343,True
31448,28354923597842736,64902.493426,True
12005,31813139061954413,9390.984939,True


In [581]:
df_feat = features_v1.sort_values(ascending=True,by='ref_hash')
df_feat.head()

,cantidad_de_subastas_en_ventana,cantidad_subastas_Monday,cantidad_subastas_Tuesday,cantidad_subastas_Wednesday,cantidad_subastas_Thursday,cantidad_subastas_Friday,cantidad_subastas_Saturday,cantidad_subastas_Sunday,ref_hash,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_14,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23
941,4.0,1.0,0.0,73.0,8.0,2.0,0.0,0.0,17411113585415081,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0
401,254.0,33.0,11.0,0.0,42.0,108.0,104.0,39.0,18336104939494418,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
980,36.0,19.0,86.0,51.0,111.0,64.0,34.0,82.0,24003528264893641,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
913,318.0,0.0,0.0,2.0,81.0,177.0,61.0,37.0,28354923597842736,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0
505,32.0,2.0,0.0,0.0,29.0,4.0,1.0,1.0,31813139061954413,1.0,...,0.0,0.0,0.0,0.0,0.0,5.0,20.0,11.0,0.0,0.0


In [582]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
x_train, x_test, y_train, y_test = train_test_split(df_feat.drop(columns='ref_hash'),
                                                    df['st'],
                                                    test_size=0.2,
                                                    random_state=43)

In [583]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

In [584]:
neigh.fit(x_train,y_train) #Fit the model using X as training data and Y as target values

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance')

In [585]:
predicted = neigh.predict(x_test) #Predict the target for the provided data

In [586]:
neigh.score(x_train, y_train)

1.0

In [587]:
neigh.score(x_test, y_test)

0.3080919428898673

# Gradient Boosting

In [104]:
from sklearn.ensemble import GradientBoostingRegressor

gradient_boost = GradientBoostingRegressor()

In [105]:
gradient_boost.fit(x_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [116]:
gradient_boost.score(x_train, y_train)

0.6676389678264849

In [106]:
gradient_boost.score(x_test, y_test)

0.5387903384814807

# Random Forest

In [588]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor()

In [589]:
random_forest.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [590]:
random_forest.score(x_train, y_train)

0.9272201120333222

In [591]:
random_forest.score(x_test,y_test)

0.48958061239983686

# Linear Regression

In [592]:
from sklearn.linear_model import LinearRegression

reg_lineal = LinearRegression()

In [593]:
reg_lineal.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [594]:
reg_lineal.score(x_train,y_train)

0.20714020618407492

In [595]:
reg_lineal.score(x_test,y_test)

0.11712444838813907

# Ada Boost

In [596]:
from sklearn.ensemble import AdaBoostRegressor

ada_boost = AdaBoostRegressor()

In [597]:
ada_boost.fit(x_train,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [598]:
ada_boost.score(x_train, y_train)

0.4355011018791608

In [599]:
ada_boost.score(x_test,y_test)

0.24130887255351421

# Decision Tree

In [600]:
from sklearn.tree import DecisionTreeRegressor

arbol_dec = DecisionTreeRegressor()

In [601]:
arbol_dec.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [602]:
arbol_dec.score(x_train,y_train)

1.0

In [603]:
arbol_dec.score(x_test,y_test)

0.16295411271148652

In [604]:
feature_importances = pd.DataFrame(arbol_dec.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(10)

,importance
cantidad_subastas_Thursday,0.573375
cantidad_subastas_Friday,0.053397
cantidad_subastas_Saturday,0.045373
cantidad_clicks_Saturday,0.034322
cantidad_subastas_Sunday,0.029143
cantidad_de_subastas_en_ventana,0.025121
cantidad_subastas_Tuesday,0.024225
cantidad_subastas_Monday,0.022527
cantidad_de_eventos_en_ventana,0.012708
cantidad_clicks_Thursday,0.012116


# XGBoost

In [605]:
from xgboost import XGBRegressor

xgboost = XGBRegressor()

ModuleNotFoundError: No module named 'xgboost'

In [140]:
xgboost.fit(x_train,y_train)

[03:17:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [141]:
xgboost.score(x_train,y_train)

0.6631303454779043

In [142]:
xgboost.score(x_test,y_test)

0.5419874635560339

# GradientBoostingSurvivalAnalysis

In [8]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

model = GradientBoostingSurvivalAnalysis()

In [9]:
model.fit(features_v1,label1_b)

GradientBoostingSurvivalAnalysis(criterion='friedman_mse', dropout_rate=0.0,
                 learning_rate=0.1, loss='coxph', max_depth=3,
                 max_features=None, max_leaf_nodes=None,
                 min_impurity_decrease=0.0, min_impurity_split=None,
                 min_samples_leaf=1, min_samples_split=2,
                 min_weight_fraction_leaf=0.0, n_estimators=100,
                 presort='auto', random_state=None, subsample=1.0,
                 verbose=0)

In [11]:
model.score(features_v2, label2_b)

0.7206307636101829